In [ ]:
import os
import random
import logging
from moviepy.editor import (
    VideoFileClip, AudioFileClip, CompositeAudioClip, concatenate_videoclips
)
from moviepy.audio.fx.all import audio_loop
from moviepy.video.fx.speedx import speedx

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def get_random_file(directory: str, extension: str) -> str:
    """
    Return a random file with the given extension from the specified directory.
    """
    files = [f for f in os.listdir(directory) if f.lower().endswith(extension)]
    if not files:
        raise FileNotFoundError(f"No file with extension {extension} found in {directory}")
    return os.path.join(directory, random.choice(files))

def adjust_audio_speed(audio_clip, target_duration):
    """
    Adjust the audio clip's speed so its duration becomes exactly target_duration.
    This function calculates the necessary speed factor (without limiting it)
    and then trims or loops the adjusted audio if needed.
    """
    current_duration = audio_clip.duration
    if current_duration == target_duration:
        return audio_clip

    # Calculate speed factor; speeding up if narration is longer than needed,
    # or slowing down if it's too short.
    speed_factor = current_duration / target_duration
    adjusted = audio_clip.fx(speedx, speed_factor)

    # Ensure the adjusted audio exactly matches target_duration
    if adjusted.duration < target_duration:
        adjusted = audio_loop(adjusted, duration=target_duration)
    elif adjusted.duration > target_duration:
        adjusted = adjusted.subclip(0, target_duration)
    return adjusted

def assemble_video(i: int):
    """
    Assemble a final video by unifying:
      - a random opening clip (from Storage/Openings/)
      - a middle video (from Storage/temp_videos/ as book_{i}.mp4)
      - a random ending clip (from Storage/Endings/)
    It overlays:
      - Background music (from Storage/music_library/song_{i}.mp3) throughout the whole video at reduced volume.
      - Narration (from Storage/temp_audios/voice_{i}.wav) that starts after the opening,
        adjusted to exactly cover the combined duration of the middle and ending clips.
    The final output is saved in Video/final_videos/ as video_{i}.mp4.
    """
    # Define directories
    dirs = {
        "openings": "Storage/Openings/",
        "middle_videos": "Storage/temp_videos/",
        "narrations": "Storage/temp_audios/",
        "music": "Storage/music_library/",
        "endings": "Storage/Endings/",
        "final_videos": "Video/final_videos/"
    }

    # Ensure the output directory exists
    os.makedirs(dirs["final_videos"], exist_ok=True)

    try:
        # Construct file paths
        middle_video_path = os.path.join(dirs["middle_videos"], f"book_{i}.mp4")
        narration_path = os.path.join(dirs["narrations"], f"voice_{i}.wav")
        song_path = os.path.join(dirs["music"], f"song_{i}.mp3")
        opening_path = get_random_file(dirs["openings"], ".mp4")
        ending_path = get_random_file(dirs["endings"], ".mp4")

        # Load video clips (assumed to be already matching in resolution & fps)
        opening_clip = VideoFileClip(opening_path)
        middle_clip = VideoFileClip(middle_video_path)
        ending_clip = VideoFileClip(ending_path)

        # Calculate durations
        t_opening = opening_clip.duration
        t_middle = middle_clip.duration
        t_ending = ending_clip.duration
        total_duration = t_opening + t_middle + t_ending
        narration_target_duration = t_middle + t_ending  # Narration spans middle + ending

        # Load audio clips
        narration_audio = AudioFileClip(narration_path)
        song_audio = AudioFileClip(song_path)

        # Adjust narration speed to exactly match the duration for middle and ending parts.
        adjusted_narration = adjust_audio_speed(narration_audio, narration_target_duration)

        # Process background music: loop it for the entire video duration and reduce volume.
        music_audio = audio_loop(song_audio, duration=total_duration).volumex(0.2)

        # Prepare audio for each segment:
        # 1. Opening uses only the background music.
        opening_audio = music_audio.subclip(0, t_opening)

        # 2. Middle segment overlays the first part of the narration over the music.
        middle_music = music_audio.subclip(t_opening, t_opening + t_middle)
        middle_narration = adjusted_narration.subclip(0, t_middle)
        middle_audio = CompositeAudioClip([middle_music, middle_narration])

        # 3. Ending segment overlays the remaining narration over the music.
        ending_music = music_audio.subclip(t_opening + t_middle, total_duration)
        ending_narration = adjusted_narration.subclip(t_middle, narration_target_duration)
        ending_audio = CompositeAudioClip([ending_music, ending_narration])

        # Set the new audio tracks (discarding any native audio from the clips)
        opening_clip = opening_clip.set_audio(opening_audio)
        middle_clip = middle_clip.set_audio(middle_audio)
        ending_clip = ending_clip.set_audio(ending_audio)

        # Concatenate the video clips (using "chain" to simply join them without extra processing)
        final_clip = concatenate_videoclips([opening_clip, middle_clip, ending_clip], method="chain")
        final_video_path = os.path.join(dirs["final_videos"], f"video_{i}.mp4")

        # Write the final video file to disk
        final_clip.write_videofile(
            final_video_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="temp-audio.m4a",
            remove_temp=True
        )
        logging.info(f"Video saved to {final_video_path}")

        # Close resources
        opening_clip.close()
        middle_clip.close()
        ending_clip.close()
        narration_audio.close()
        song_audio.close()
        final_clip.close()

    except Exception as e:
        logging.error(f"An error occurred during video assembly for book {i}: {e}")

def main():
    """
    Process videos for 5 books, handling any potential errors for individual books.
    """
    # Ensure directories exist
    os.makedirs("Storage/temp_videos", exist_ok=True)
    os.makedirs("Storage/temp_audios", exist_ok=True)
    os.makedirs("Storage/music_library", exist_ok=True)
    os.makedirs("Video/final_videos", exist_ok=True)

    # Process 5 books
    for book_number in range(1, 6):
        try:
            logging.info(f"Processing video for book {book_number}")
            assemble_video(book_number)
        except Exception as e:
            logging.error(f"Failed to process video for book {book_number}: {e}")

if __name__ == '__main__':
    main()
